In [1]:
import pandas as pd
import numpy as np
import datetime
import math
from gensim import corpora, models
from pyltp import SentenceSplitter
import functools
import os
from pyltp import Segmentor
from pyltp import Postagger
import jieba
import jieba.posseg as psg
from jieba import analyse
import re



In [64]:
all_docs_df = pd.read_csv('../../data/chusai/all_docs.txt', sep='\001', header=None)
all_docs_df.columns = ['id', 'title', 'text']
all_docs_df['title'] = all_docs_df['title'].astype(str)
all_docs_df['text'] = all_docs_df['text'].astype(str)
print(all_docs_df.info())
print(all_docs_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108295 entries, 0 to 108294
Data columns (total 3 columns):
id       108295 non-null object
title    108295 non-null object
text     108295 non-null object
dtypes: object(3)
memory usage: 2.5+ MB
None
        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...  
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...  
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...  
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...  
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...  


In [65]:
train_doc_keyword_df = pd.read_csv('../../data/chusai/train_docs_keywords.txt', sep='\t', header=None)
train_doc_keyword_df.columns = ['id', 'keyword']
train_doc_keyword_df['keyword_list'] = train_doc_keyword_df['keyword'].map(lambda x: x.split(','))
print(train_doc_keyword_df.head(30))


         id             keyword             keyword_list
0   D012650          三生三世,凤九,东华           [三生三世, 凤九, 东华]
1   D047118               南宁,美食                 [南宁, 美食]
2   D098970              凉菜,北京人                [凉菜, 北京人]
3   D092010           华为,P30pro             [华为, P30pro]
4   D103408              酒吧,世界杯                [酒吧, 世界杯]
5   D103809             烧烤,人生一串               [烧烤, 人生一串]
6   D003823          惠若琪,杨颖,奔跑吧           [惠若琪, 杨颖, 奔跑吧]
7   D023499   我与你的光年距离2,王以纶,许晓诺    [我与你的光年距离2, 王以纶, 许晓诺]
8   D063937           无人机,军用,美军            [无人机, 军用, 美军]
9   D059085            女兵,剪影,欧美             [女兵, 剪影, 欧美]
10  D045459         增肌,器械,哑铃,杠铃         [增肌, 器械, 哑铃, 杠铃]
11  D069093         支教,艺术,乡村,儿童         [支教, 艺术, 乡村, 儿童]
12  D082165             第三世界,游戏               [第三世界, 游戏]
13  D071067               蛋糕,宝宝                 [蛋糕, 宝宝]
14  D004539         纪念品,蒂尔达·斯文顿           [纪念品, 蒂尔达·斯文顿]
15  D057452        叙利亚,俄罗斯,S300         [叙利亚, 俄罗斯, S300]
16  D008361               王源,微博

In [66]:
#给jieba添加自定义词
for keyword_list in train_doc_keyword_df['keyword_list']:
    for keyword in keyword_list:
        jieba.add_word(keyword)

jieba.load_userdict('user_dict.txt')


In [67]:
LTP_DATA_DIR = '../../ltp/ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型

pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型



In [68]:
#停用词表加载方法
def get_stopword_list():
    #停用词表存储路径，每一行为一个词，按行读取进行加载
    #进行编码转换确保匹配准确率
    stop_word_path = '../stopword.txt'
    stop_word_list = [sw.replace('\n', '') for sw in open(stop_word_path).readlines()]
    return stop_word_list

#分词方法，调用结巴接口
def jieba_seg_to_list(sentence, pos=False):
    if not pos:
        #不进行词性标注的分词方法
        seg_list = jieba.cut(sentence)
    else:
        #进行词性标注的分词方法
        seg_list = psg.cut(sentence)
    return seg_list

#去除干扰词
def jieba_word_filter(seg_list, stopword_list, pos=False):
    
    filter_list = []
    #根据pos参数选择是否词性过滤
    #不进行词性过滤，则将词性都标记为n，表示全部保留
    for seg in seg_list:
        if not pos:
            word = seg
            flag = 'n'
        else:
            word = seg.word
            flag = seg.flag
        if not flag.startswith('n'):
            continue
        #过滤高停用词表中的词，以及长度为<2的词
        if not word in stopword_list and len(word) > 1:
            filter_list.append(word)
    
    return filter_list

def jieba_word_deal(sentence, stopword_list, pos=False):
    #调用上面方式对数据集进行处理，处理后的每条数据仅保留非干扰词
    seg_list = jieba_seg_to_list(sentence, pos)
    filter_list = jieba_word_filter(seg_list, stopword_list, pos)
    return filter_list

def jieba_title_word_n(sentence, stopword_list):
    #调用上面方式对数据集进行处理，处理后的每条数据仅保留非干扰词
    seg_list = jieba_seg_to_list(sentence, True)
    title_word_n_list = []
    for seg in seg_list:
        word = seg.word
        flag = seg.flag
        if flag.startswith('n'):
            title_word_n_list.append(word)
    return title_word_n_list

#分词方法，调用ltp接口
def ltp_seg_to_list(sentence):
    words = segmentor.segment(sentence)  # 分词
    seg_list = list(words)
    return seg_list

#去除干扰词
def ltp_word_filter(seg_list, stopword_list):
    
    filter_list = []
    #根据pos参数选择是否词性过滤
    #不进行词性过滤，则将词性都标记为n，表示全部保留
    for seg in seg_list:
        #过滤高停用词表中的词，以及长度为<2的词
        if not seg in stopword_list and len(seg) > 1:
            filter_list.append(seg)
    
    return filter_list

def ltp_word_deal(sentence, stopword_list):
    #调用上面方式对数据集进行处理，处理后的每条数据仅保留非干扰词
    seg_list = ltp_seg_to_list(sentence)
    filter_list = ltp_word_filter(seg_list, stopword_list)
    return filter_list

def get_title_person_name(title_list):
    attributes_list = list(postagger.postag(title_list))
    title_person_name_list = list()
    i = 0
    for attributes in attributes_list:
        if attributes == 'nh':
            title_person_name_list.append(title_list[i])
        i = i + 1
    return title_person_name_list

def get_text_sentences(text):
    sentences_list = SentenceSplitter.split(text)
    return sentences_list

all_docs_df['text_sentences'] = all_docs_df['text'].map(lambda x: get_text_sentences(x))
all_docs_df['text_sentences_len'] = all_docs_df['text_sentences'].map(lambda x: len(x))

def get_title_text(all_docs_df):
    temp_df = pd.DataFrame(columns=['id', 'title_text'])
    for temp_id, title, text, text_sentences_len in all_docs_df[['id', 'title', 'text', 'text_sentences_len']].values:
        length = math.ceil(text_sentences_len * 0.4)
        if length < 6:
            length = 6
#         if length > 15:
#             length = 15
        title_text = ''
        for i in range(length):
            title_text = title + '。' + title_text
        title_text = title_text + text
        temp = pd.DataFrame([[temp_id, title_text]], columns=['id', 'title_text'])
        temp_df = pd.concat([temp_df, temp])
    all_docs_df = pd.merge(all_docs_df, temp_df, on='id', how='left')
    return all_docs_df

stopword_list = get_stopword_list()
all_docs_df['jieba_title_list'] = all_docs_df['title'].map(lambda x : jieba_word_deal(x, stopword_list, False))
all_docs_df['jieba_title_word_n_list'] = all_docs_df['title'].map(lambda x : jieba_title_word_n(x, stopword_list))
# all_docs_df['jieba_title_list_for_name'] = all_docs_df['title'].map(lambda x : jieba_word_deal(x, stopword_list, False))
all_docs_df['title_text'] =  all_docs_df['title'] + '。' + all_docs_df['text']
# all_docs_df = get_title_text(all_docs_df)
all_docs_df['jieba_title_text_list'] = all_docs_df['title_text'].map(lambda x : jieba_word_deal(x, stopword_list, False))

all_docs_df['ltp_title_list'] = all_docs_df['title'].map(lambda x : ltp_word_deal(x, stopword_list))
all_docs_df['ltp_title_text_list'] = all_docs_df['title_text'].map(lambda x : ltp_word_deal(x, stopword_list))

all_docs_df['jieba_title_person_name_list'] = all_docs_df['jieba_title_list'].map(lambda x: get_title_person_name(x))
all_docs_df['ltp_title_person_name_list'] = all_docs_df['ltp_title_list'].map(lambda x: get_title_person_name(x))

print(all_docs_df.head(10))


        id                   title  \
0  D000001      林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002      小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003       年轻时的她风华绝代，现却无人送祝福   
3  D000004      林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     曾是TVB颜值担当，近照曝光发现真老了   
5  D000006  16岁的范冰冰有多美？塞娅公主看一眼被惊艳到   
6  D000007        一样的北京，不一样的周迅与周冬雨   
7  D000008      张杰自嘲是废物，赞谢娜是新时代好女性   
8  D000009    杨幂获杂志主编夸奖：大幂幂瘦了越来越漂亮   
9  D000010       杨幂女扮男装现身新节目瞬间年轻十岁   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   
5  范冰冰，家喻户晓的人物了，在出演还珠格格之前已经亭亭玉立十分美丽，在一些影剧中饰演一些小角色...   
6  不知道刘若英与陈可辛之间有怎样的交情。《后来的我们》首映请来了陈可辛，他看完后笑着说很羡慕刘...   
7  本文是企鹅号"不八卦会死人"原创文章，独家发布企鹅号，未经允许，不得转载!最新一期《向往的生...   
8  网易娱乐4月29日报道  4月29日，时尚主编张宇晒出一组与杨幂的自拍合影，并发文：“今天与...   
9  杨幂前段时间录制的综艺节目《我是大侦

In [69]:
#idf统计方法
def train_idf(doc_list):
    idf_dic = {}
    #总文档数
    tt_count = len(doc_list)
    
    #每个词出现的文档数
    for doc in doc_list:
        for word in set(doc):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
    
    #按公式转换为idf值，分母加1进行平滑处理
    for k, v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (1.0 + v))
        
    #对于没有在字典中的词，默认其尽在一个文档中出现，得到默认idf值
    default_idf = math.log(tt_count / (1.0))
    return idf_dic, default_idf

#排序函数，用于topK关键词的按值排
def cmp(e1, e2):
    res = np.sign(e1[1] - e2[1])
    if res != 0:
        return res
    else:
        a = e1[0] + e2[0]
        b = e2[0] + e1[0]
        if a > b:
            return 1
        elif a == b:
            return 0
        else:
            return -1
        


In [70]:
#TF-IDF类
class TfIdf(object):
    #四个参数分别是：训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
    def __init__(self, idf_dic, default_idf, word_list, keyword_num):
        self.word_list = word_list
        self.idf_dic = idf_dic
        self.default_idf = default_idf
        self.tf_dic = self.get_tf_dic()
        self.keyword_num = keyword_num
    
    #统计tf值
    def get_tf_dic(self):
        tf_dic = {}
        for word in self.word_list:
            tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
        
        tt_count = len(self.word_list)
        for k, v in tf_dic.items():
            tf_dic[k] = float(v) / tt_count
        
        return tf_dic
    
    #按公式计算tf-idf
    def get_tfidf(self):
        tfidf_dic = {}
        for word in self.word_list:
            idf = self.idf_dic.get(word, self.default_idf)
            tf = self.tf_dic.get(word, 0)
            
            tfidf = tf * idf
            tfidf_dic[word] = tfidf
        
        result_dict = {}
        for k, v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
            result_dict[k] = result_dict.get(k, 0.0) + float(v)
        return result_dict



In [71]:
jieba_idf_dic, jieba_default_idf = train_idf(all_docs_df['jieba_title_text_list'])
all_docs_df['jieba_result_dict_5'] = all_docs_df['jieba_title_text_list'].map(lambda x: TfIdf(jieba_idf_dic, jieba_default_idf, x, 5).get_tfidf())
all_docs_df['jieba_result_dict_10'] = all_docs_df['jieba_title_text_list'].map(lambda x: TfIdf(jieba_idf_dic, jieba_default_idf, x, 10).get_tfidf())
print(all_docs_df.head())


        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                                      text_sentences  text_sentences_len  \
0  [早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格...                  25   
1  [翩若惊鸿，婉若游龙。, 曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。, 直...                   8   
2  [上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的...                   7   
3  [霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称...                   9   
4  [不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。, 可以...                  27

In [72]:
ltp_idf_dic, ltp_default_idf = train_idf(all_docs_df['ltp_title_text_list'])
all_docs_df['ltp_result_dict_5'] = all_docs_df['ltp_title_text_list'].map(lambda x: TfIdf(ltp_idf_dic, ltp_default_idf, x, 5).get_tfidf())
all_docs_df['ltp_result_dict_10'] = all_docs_df['ltp_title_text_list'].map(lambda x: TfIdf(ltp_idf_dic, ltp_default_idf, x, 10).get_tfidf())
print(all_docs_df.head())


        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                                      text_sentences  text_sentences_len  \
0  [早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格...                  25   
1  [翩若惊鸿，婉若游龙。, 曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。, 直...                   8   
2  [上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的...                   7   
3  [霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称...                   9   
4  [不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。, 可以...                  27

In [73]:
sample_df = pd.read_csv('../../result/chusai/sample.csv', encoding='ISO-8859-1')
print(len(sample_df))
sample_df = pd.merge(sample_df, all_docs_df, on='id', how='left')
print(sample_df.head())


107295
        id label1 label2                title  \
0  D000001     ÎÞ     ÎÞ   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002     ÎÞ     ÎÞ   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003     ÎÞ     ÎÞ    年轻时的她风华绝代，现却无人送祝福   
3  D000004     ÎÞ     ÎÞ   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     ÎÞ     ÎÞ  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                                      text_sentences  text_sentences_len  \
0  [早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格...                  25   
1  [翩若惊鸿，婉若游龙。, 曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。, 直...                   8   
2  [上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的...                   7   
3  [霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称...     

In [74]:
def is_contain_alpha(x):
    my_re = re.compile(r'[A-Za-z]',re.S)
    res = re.findall(my_re,x)
    if len(res):
        return True
    else:
        return False

keyword_set = set()
for keyword_list in train_doc_keyword_df['keyword_list']:
    for keyword in keyword_list:
        keyword_set.add(keyword)

def get_deal_result_list(keyword_set, sample_df):
    temp_df = pd.DataFrame(columns=['id', 'result_list'])
    for temp_id, jieba_title_list, result_dict, jieba_title_person_name_list, ltp_title_person_name_list, title, jieba_title_word_n_list in sample_df[['id', 'jieba_title_list', 'jieba_result_dict_10', 'jieba_title_person_name_list', 'ltp_title_person_name_list', 'title', 'jieba_title_word_n_list']].values:
        show_list = list()
        word_list = re.findall(r"《(.+?)》", title)
        for word in word_list:
            if ',' in word:
                word = word.replace(',', '，')
                print(word)
            show_list.append(word)
        title_set = set(jieba_title_list)
        keys = list(result_dict.keys())
        title_person_name_set = set(jieba_title_person_name_list) & set(ltp_title_person_name_list)
#         title_person_name_set = set(jieba_title_person_name_list)
        jieba_title_word_n_set = set(jieba_title_word_n_list)
        result_list = list()
        result_list_1 = list()
        result_list_2 = list()
        result_list_3 = list()
        result_list_4 = list()
        result_list_5 = list()
        result_list_6 = list()
        result_list_7 = list()
        for title in title_set:
            if len(title) >= 5:
                result_list.append(title)
        for key in keys:
            if key in set(result_list):
                continue
            if((key in keyword_set) & (key in title_set)):
                result_list_1.append(key)
            else:
                if key in title_person_name_set:
                    result_list_5.append(key)
                else:
                    if (((key in set(jieba_title_person_name_list)) | (key in set(ltp_title_person_name_list))) & (len(key) >= 3)):
                        continue
                    else:
#                         if is_contain_alpha(key):
#                             result_list_7.append(key)
#                         else:
                        if key in title_set:
                            if key in jieba_title_word_n_set:
                                result_list_7.append(key)
                            else:
                                result_list_3.append(key)
                        else:
                            if key in keyword_set:
                                result_list_2.append(key)
                            else:
                                result_list_4.append(key)
        for name in set(jieba_title_person_name_list):
            if name in set(result_list_5):
                continue
            else:
                if len(name) >= 3:
                    result_list_6.append(name)
#         for name in set(ltp_title_person_name_list):
#             if ((name in set(result_list_5)) | (name in set(result_list_6))):
#                 continue
#             else:
#                 if len(name) >= 3:
#                     result_list_6.append(name)
#         result_list = result_list_1 + result_list + show_list + result_list_5 + result_list_6 + result_list_3 + result_list_2 + result_list_4
        result_list = show_list + result_list_5 + result_list_1 + result_list + result_list_6 + result_list_7 + result_list_3 + result_list_2 + result_list_4
        final_list = list()
        for result in result_list:
            if result not in final_list:
                final_list.append(result)
        temp = pd.DataFrame([[temp_id, final_list]], columns=['id', 'result_list'])
        temp_df = pd.concat([temp_df, temp])
    print(temp_df.head())
    sample_df = pd.merge(sample_df, temp_df, on='id', how='left')
    return sample_df

def get_top_n_word(result_list, n):
    if len(result_list) < n:
        return '无'
    else:
        return result_list[n - 1]
    
sample_df = get_deal_result_list(keyword_set, sample_df)
sample_df['label1'] = sample_df['result_list'].map(lambda x: get_top_n_word(x, 1))
sample_df['label2'] = sample_df['result_list'].map(lambda x: get_top_n_word(x, 2))
print(sample_df.head())


你好，之华
思索，4色
        id                                        result_list
0  D000001      [林志颖, 睫毛, 面膜, 陈若仪, 日常, 安利, 爱用, 洁面, 很像种, 小零小碎]
0  D000002     [张钧甯, 洛神, 姿艳逸, 外朗, 仪静体闲, 皓齿, 丹唇, 内鲜, 婉若, 翩若惊鸿]
0  D000003  [送祝福, 风华绝代, 生日, 蓝洁瑛, 妩媚动人, 美得, 最火时, 春三十娘, 家有娇妻...
0  D000004        [霍建华, 老公, 林心如, 粉丝, 女儿, 索吻, 霍粉, 就会开, 再亲, 提及]
0  D000005         [TVB, 颜值, 素颜, 姚莹莹, 药房, 剧集, 感概, 行踪, 拍到, 节俭]
        id label1 label2                title  \
0  D000001    林志颖     睫毛   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002    张钧甯     洛神   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    送祝福   风华绝代    年轻时的她风华绝代，现却无人送祝福   
3  D000004    霍建华     老公   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005    TVB     颜值  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧

In [77]:
xianshang_df = pd.read_csv('../../result/chusai/tfidf_10_10_1.csv')

sample_df['xianshang_label1'] = xianshang_df['label1']
sample_df['label1_true'] = (sample_df['label1'] == sample_df['xianshang_label1'])
print(len(sample_df[sample_df.label1_true != True]))
print(sample_df[['label1', 'xianshang_label1']].head())

# xianshang_df['length6_label1'] = sample_df['label1']
# xianshang_df['length6_label2'] = sample_df['label2']

# temp_df = pd.DataFrame(columns=['id', 'result_list'])
# for label1, label2, length6_label1, length6_label2, instance_id in xianshang_df[['label1', 'label2', 'length6_label1', 'length6_label2', 'id']].values:
#     result_list = [length6_label1, label1, label2, length6_label2]
#     final_list = list()
#     for result in result_list:
#         if result not in final_list:
#             final_list.append(result)
#     temp = pd.DataFrame([[instance_id, final_list]], columns=['id', 'result_list'])
#     temp_df = pd.concat([temp_df, temp])

# def get_top_n_word(result_list, n):
#     if len(result_list) < n:
#         return '无'
#     else:
#         return result_list[n - 1]

# temp_df['label1'] = temp_df['result_list'].map(lambda x: get_top_n_word(x, 1))
# temp_df['label2'] = temp_df['result_list'].map(lambda x: get_top_n_word(x, 2))




26066
  label1 xianshang_label1
0    林志颖              林志颖
1    张钧甯               杨幂
2    送祝福               年轻
3    霍建华              霍建华
4    TVB              TVB


In [78]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../result/chusai/%s.csv' % fileName, header=True, index=False)
    
exportResult(sample_df[['id', 'label1', 'label2']], 'length1')

